In [4]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 131352 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.1-0ubuntu3~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.1-0ubuntu3~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.1-0ubuntu3~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=ht

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [6]:
%cd /content/drive/defect-detection

/content/drive/defect-detection


In [0]:
!pip install -r requirements.txt

In [0]:
import os
import shutil
from tqdm import tqdm

neg_files = os.listdir("Class6/neg")
pos_files = os.listdir("Class6/pos")

neg = len(neg_files)
pos = len(pos_files)

#%80 train %20 test for negative images
neg_train = int(neg * 0.8)
neg_test = neg - neg_train
#%80 train %20 test for positive images
pos_train = int(pos * 0.8)
pos_test = pos - pos_train

In [0]:
os.makedirs('train/negative')
os.makedirs('train/positive')
os.makedirs('test/negative')
os.makedirs('test/positive')

In [0]:
for i in tqdm(range(0, neg_train)):
    file = "Class6/neg/" + neg_files[i]
    shutil.move(file, "train/negative")

100%|██████████| 66/66 [02:06<00:00,  2.16s/it]


In [0]:
for i in tqdm(range(neg_train, neg)):
    file = "Class6/neg/" + neg_files[i]
    shutil.move(file, "test/negative")

100%|██████████| 17/17 [00:32<00:00,  1.89s/it]


In [0]:
for i in tqdm(range(0, pos_train)):
    file = "Class6/pos/" + pos_files[i]
    shutil.move(file, "train/positive")

100%|██████████| 393/393 [12:18<00:00,  1.81s/it]


In [0]:
for i in tqdm(range(pos_train, pos)):
    file = "Class6/pos/" + pos_files[i]
    shutil.move(file, "test/positive")

100%|██████████| 99/99 [02:59<00:00,  1.86s/it]


In [0]:
import numpy as np
import pandas as pd
from imutils import paths

from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout


In [0]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [0]:
train_data_dir = "train"
test_data_dir = "test"
TRAIN = len(list(paths.list_images(train_data_dir)))
TEST = len(list(paths.list_images(test_data_dir)))
BS = 8
EPOCHS = 5
img_width, img_height = 512, 512

In [0]:
classifier = Sequential()

classifier.add(Conv2D(128, (2,2), 
                      input_shape = (img_width, img_height, 3), 
                      activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

classifier.add(Conv2D(64, (3,3), activation = 'relu'))
classifier.add(Conv2D(64, (2,2), activation = 'relu'))
classifier.add(Conv2D(64, (1,1), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

classifier.add(Conv2D(32, (2,2), activation = 'relu'))
classifier.add(Conv2D(32, (2,2), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

classifier.add(Flatten())

classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dropout(0.3))
classifier.add(Dense(units = 10, activation = 'relu'))
classifier.add(Dropout(0.3))
classifier.add(Dense(units = 1, activation = 'sigmoid'))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
classifier.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 511, 511, 128)     1664      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 255, 255, 128)     0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 253, 253, 64)      73792     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 252, 252, 64)      16448     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 252, 252, 64)      4160      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 126, 126, 64)      0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 125, 125, 32)      8224      
__________

In [0]:
# compile model
classifier.compile(optimizer = 'adam', 
                   loss = 'binary_crossentropy', 
                   metrics = ['accuracy'])

In [0]:
# create data generators and data pathways
from keras.preprocessing.image import ImageDataGenerator

trainAug = ImageDataGenerator(rescale = 1./255,
                               shear_range = 0.2,
                               zoom_range = 0.2,
                               horizontal_flip = True,
                               fill_mode = "nearest")

testAug = ImageDataGenerator(rescale = 1./255,
                             fill_mode = "nearest")

trainGen = trainAug.flow_from_directory('train',
                                         target_size = (img_width, img_height),
                                         batch_size = BS,
                                         class_mode = 'binary')

testGen = testAug.flow_from_directory('test',
                                    target_size = (img_width, img_height),
                                    batch_size = BS,
                                    class_mode = 'binary')

Found 459 images belonging to 2 classes.
Found 116 images belonging to 2 classes.


In [0]:
classes = trainGen.class_indices    
print(classes)

{'negative': 0, 'positive': 1}


In [0]:
# train model
H = classifier.fit_generator(trainGen,
                             epochs = EPOCHS,
                             steps_per_epoch = TRAIN // BS)

Epoch 1/5
57/57 [==============================] - 1217s 21s/step - loss: 0.6125 - acc: 0.8333
Epoch 2/5
57/57 [==============================] - 1196s 21s/step - loss: 0.4339 - acc: 0.8596
Epoch 3/5
57/57 [==============================] - 1197s 21s/step - loss: 0.1571 - acc: 0.9342
Epoch 4/5
57/57 [==============================] - 1187s 21s/step - loss: 0.0345 - acc: 0.9759
Epoch 5/5
57/57 [==============================] - 1179s 21s/step - loss: 0.0190 - acc: 0.9789


In [0]:
#save model
classifier.save('model.h5')

In [24]:
from keras.models import load_model

classifier = load_model('model.h5')
classifier.compile(optimizer = 'adam',
                  loss = 'binary_crossentropy',
                  metrics =['accuracy'])

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.


In [60]:
import cv2
import numpy as np

img_width, img_height = 512, 512 
directory = 'Class6/Test'

for file in os.listdir(directory):
  filename = os.path.join(directory, file)
  if filename.endswith(".PNG"):
    img = cv2.imread(filename)
    img= cv2.resize(img, (img_width, img_height))
    img = img.astype("float")/255.0
    img = np.reshape(img, [1, img_width, img_height, 3])
    
    pred = classifier.predict(img)
    predicted_class = "UNRECOGNIZABLE"
    if (pred <= 0.5):
      predicted_class = "Defect"
    else:
      predicted_class = "No defect"
    
    print("Predicted class for: " + file + " is => " + predicted_class)
    
  else:
    print("The directory " + filename + " could not be located.")
    
    
    

Predicted class for: 0478.PNG is => Defect
Predicted class for: 0485.PNG is => Defect
Predicted class for: 0482.PNG is => No defect
Predicted class for: 0477.PNG is => No defect
Predicted class for: 0476.PNG is => No defect
Predicted class for: 0486.PNG is => No defect
Predicted class for: 0484.PNG is => No defect
Predicted class for: 0479.PNG is => No defect
Predicted class for: 0480.PNG is => No defect
Predicted class for: 0481.PNG is => Defect
Predicted class for: 0483.PNG is => No defect
Predicted class for: 0487.PNG is => No defect
Predicted class for: 0495.PNG is => Defect
Predicted class for: 0497.PNG is => No defect
Predicted class for: 0496.PNG is => No defect
Predicted class for: 0491.PNG is => No defect
Predicted class for: 0498.PNG is => No defect
Predicted class for: 0494.PNG is => No defect
Predicted class for: 0492.PNG is => Defect
Predicted class for: 0490.PNG is => No defect
Predicted class for: 0493.PNG is => No defect
Predicted class for: 0488.PNG is => No defect
Pre